<a href="https://colab.research.google.com/github/PinkPigmyPuff/SeatingChart/blob/main/ClassroomGA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [96]:
import random
import numpy as np
from typing import List, Optional, Callable, Tuple


Chromosome = List[int]
Population = List[Chromosome]
Opinion = List[int]

In [97]:
def genChromosome(size: int) -> Chromosome:
    return random.sample(range(0, size), size)

def genPopulation(popSize: int, chromeosomeSize: int) -> Population:
    return [genChromosome(chromeosomeSize) for _ in range(popSize)]

def genOpinions(chromosome: Chromosome, opinionMax: int) -> Opinion:
  likes = []
  dislikes = []
  # for every student, randomly generate how many students they have opionions on, and add an empty array to like/dislike
  for i in range(0, len(chromosome)):
    likes.append([])
    dislikes.append([])
    likeNum = random.randint(0, opinionMax)
    dislikeNum = random.randint(0, opinionMax)
    dupelist = chromosome.copy()
    dupelist.pop(i)
    
    # choose random students that the current student likes
    for j in range(0, likeNum):
      randomStudent = random.randrange(0, len(dupelist))
      likes[i].append(dupelist[randomStudent])
      dupelist.pop(randomStudent)

    # choose random students that the current student dislikes
    for j in range(0, dislikeNum):
      randomStudent = random.randrange(0, len(dupelist))
      dislikes[i].append(dupelist[randomStudent])
      dupelist.pop(randomStudent)

  return likes, dislikes

In [98]:
def fitness(chromosome: Chromosome, peoplePerTable: int = 4, nextToFriendCost: int = 2, nextToEnemyCost: int = -1) -> int:
    size = len(chromosome)
    if size != len(likes) != len(dislikes):
        raise ValueError("genome and likes/dislikes must be of the same length")

    if size % peoplePerTable != 0:
        raise ValueError("The length of the chromosome must be divisable by peoplePerTable")
    # determine the cost of the given chromosome
    peopleHappiness = [0] * size
    chunkedClassroom = chunks(chromosome, peoplePerTable)
    # for every student
    for student in range(0, len(chromosome)):
      currentStudent = chromosome[student]
      studentsTable = 0
      # finds which table the currentStudent is sitting at
      if any(currentStudent in (match := nested_list) for nested_list in chunkedClassroom):
        studentsTable = chunkedClassroom.index(match)
        # print(f"current student is {currentStudent}, and they sit at table {studentsTable}")

      # determine the happiness of the currentStudent, based on each person at their table
      for tableMate in range(0, peoplePerTable):
        currentTableMate = chunkedClassroom[studentsTable][tableMate]
        if currentTableMate in likes[student]:
          peopleHappiness[student] += nextToFriendCost
        elif currentTableMate in dislikes[student]:
          peopleHappiness[student] += nextToEnemyCost

    return average(peopleHappiness)

def populationFitness(population: Population) -> float:
    return [fitness(chromosome) for chromosome in population]
    
# Return n-sized chunks from lst
def chunks(lst, n):
    x = [lst[i:i + n] for i in range(0, len(lst), n)]
    return(x)

def relu(x):
	  return max(0.0, x)
 
def average(lst):
   return float(sum(lst) / len(lst))

In [99]:
# Randomly chooses n individuals from the population and returns the fittest one
def tournamentSelection(population: Population, tournamentSize: int) -> Population:
		
		tournament = []
		tempPop = population.copy()
		for i in range(0, tournamentSize):
			random_id = random.randint(0, len(tempPop)-1)
			tournament.append(tempPop[random_id])
			tempPop.pop(i)

		#print(f"tournament pool! {tournament}")
		fittest = chooseHighest(tournament, 1)
		#print(f"fittest! {fittest}")
		return fittest

In [100]:
def roulette_wheel_selection(population: Population, populationFitness) -> Population:
    if len(population) != len(populationFitness):
        raise ValueError("genome and likes/dislikes must be of the same length")
    print(f"pFit: {populationFitness}")

    # Computes the totallity of the population fitness
    sumFitness = sum(populationFitness)
    print(f"sumFittness: {sumFitness}")
    
    # Computes for each chromosome the probability 
    chromosome_probabilities = [classroomHappiness/sumFitness for classroomHappiness in populationFitness]
    print(f"c_prob {chromosome_probabilities}")

    # Selects one chromosome based on the computed probabilities
    choice = population[np.random.choice(range(0, len(population)), p=chromosome_probabilities)]
    print(f"chioce_fitness {fitness(choice)}")
    return choice

In [101]:
def partiallyMatchedCrossover(ind1, ind2):
    #print(f"ind1: {ind1}\nind2: {ind2}")
    size = min(len(ind1), len(ind2))
    p1, p2 = [0] * size, [0] * size

    # Initialize the position of each indices in the individuals
    for i in range(size):
        p1[ind1[i]] = i
        p2[ind2[i]] = i
    # Choose crossover points
    cxpoint1 = random.randint(0, size)
    cxpoint2 = random.randint(0, size - 1)
    if cxpoint2 >= cxpoint1:
        cxpoint2 += 1
    else:  # Swap the two cx points
        cxpoint1, cxpoint2 = cxpoint2, cxpoint1

    # Apply crossover between cx points
    for i in range(cxpoint1, cxpoint2):
        # Keep track of the selected values
        temp1 = ind1[i]
        temp2 = ind2[i]
        # Swap the matched value
        ind1[i], ind1[p1[temp2]] = temp2, temp1
        ind2[i], ind2[p2[temp1]] = temp1, temp2
        # Position bookkeeping
        p1[temp1], p1[temp2] = p1[temp2], p1[temp1]
        p2[temp1], p2[temp2] = p2[temp2], p2[temp1]

    #print(f"FINSIHED, ind1: {ind1}\n ind2: {ind2}")
    return ind1, ind2

In [102]:
def mutate(chromosome: Chromosome) -> Chromosome:
    length = len(chromosome)
    mutatedChromosome = chromosome.copy()
    if length < 2:
      print("Chromosome is too short to be mutated")
      return mutatedChromosome
    index1, index2 = random.sample(range(0, length), 2)
    mutatedChromosome[index1], mutatedChromosome[index2] = mutatedChromosome[index2], mutatedChromosome[index1]
    #print(f"mutated chromosome: {mutatedChromosome}")
    return mutatedChromosome

In [103]:
def chooseHighest(population: Population, num: int = 3):
  #print(f"pop to choose from: {population}")
  fit = populationFitness(population)
  #print(f"pop fitness: {fit}")
  highestIndex = 0
  for i in range(0, len(fit)):
      if fit[i] > fit[highestIndex]:
          highest = fit[i]
          highestIndex = i
  #print(f"chosen: {population[highestIndex]}")
  return population[highestIndex]

In [104]:
# Generates an entirely new parent population, and shuffles it
def newPop(breedingPop: Population) -> Population:
  newPop = []
  for i in range(len(breedingPop)):
      offspring = tournamentSelection(breedingPop, 4)
      newPop.append(mutate(offspring))
  return newPop

In [105]:
pop = genPopulation(40, 40)
likes, dislikes = genOpinions(pop[0], 4)

def run_evolution(
        generation_limit: int = 30,
        population = genPopulation(8, 8), 
        #likes = genOpinions(population[0], 4),
        #dislikes = genOpinions(population[0], 4)
        #peoplePerTable = 4,
        #nextToFriendCost = 1,
        #nextToEnemyCost = -1,
        ):
    print(f"OG population: {population}")
    print(f"Likes: {likes}\nDislikes: {dislikes}")
    if len(population) % 2 != 0:
        print("Population size is odd, removing one chromosome")
        population.pop(0)


    for i in range(generation_limit):
        #random.shuffle(population)
        #parents = [tournamentSelection(population, 2) for i in range(len(population))]
        popFitness = populationFitness(population)
        print(f"\n\nGeneration {i} fitness: {popFitness}")
        print(f"population: {population}")
        ##parents = [roulette_wheel_selection(population, popFitness) for i in range(len(population))]

        population = newPop(population)

        #while parents:
            #print(f"parents: {parents}")
            #offspring1, offspring2 = partiallyMatchedCrossover(parents[0], parents[1])
            #print(f"nextPopulation CROSSED: {offspring1, offspring2}")
            #next_generation.append(mutate(offspring1))
            #next_generation.append(mutate(offspring2))
            #print(f"nextPopulation MUTATED: {next_generation}")
            #parents = parents[2:]
            
        
        #population = next_generation

    return chooseHighest(population)

run_evolution(population=pop)

OG population: [[23, 27, 32, 18, 12, 35, 34, 33, 17, 1, 38, 13, 22, 6, 29, 5, 15, 25, 28, 39, 19, 7, 20, 16, 31, 3, 9, 24, 14, 37, 2, 21, 26, 4, 30, 10, 36, 11, 8, 0], [36, 29, 26, 10, 17, 9, 0, 8, 11, 4, 3, 34, 18, 32, 33, 22, 25, 16, 37, 27, 20, 21, 23, 19, 7, 1, 5, 35, 13, 14, 31, 2, 12, 15, 39, 24, 30, 28, 38, 6], [4, 39, 17, 14, 34, 1, 7, 35, 30, 6, 20, 38, 9, 36, 26, 10, 37, 8, 28, 12, 11, 18, 16, 32, 2, 25, 22, 5, 13, 24, 29, 19, 31, 21, 33, 0, 27, 3, 15, 23], [0, 3, 20, 32, 35, 4, 38, 37, 10, 22, 27, 17, 33, 26, 13, 6, 11, 5, 15, 8, 2, 29, 14, 16, 24, 25, 19, 34, 36, 28, 12, 1, 30, 18, 7, 39, 23, 21, 9, 31], [29, 28, 32, 33, 10, 6, 25, 4, 38, 27, 20, 24, 30, 12, 35, 8, 3, 17, 18, 2, 21, 22, 13, 23, 26, 34, 16, 14, 11, 9, 15, 1, 19, 36, 37, 0, 31, 7, 5, 39], [0, 33, 13, 6, 25, 10, 21, 30, 16, 27, 12, 2, 14, 5, 23, 18, 26, 28, 34, 38, 35, 29, 7, 3, 36, 19, 39, 15, 17, 8, 22, 31, 32, 37, 24, 1, 4, 20, 11, 9], [9, 17, 3, 39, 2, 19, 35, 15, 14, 33, 38, 26, 11, 37, 5, 21, 18, 22, 34,

[15,
 37,
 28,
 22,
 32,
 12,
 7,
 13,
 36,
 4,
 31,
 6,
 35,
 26,
 19,
 2,
 5,
 21,
 30,
 23,
 9,
 14,
 27,
 38,
 18,
 11,
 29,
 1,
 17,
 0,
 10,
 34,
 25,
 20,
 16,
 3,
 24,
 8,
 39,
 33]

1. Encoding scheme
- A chromosome is a permutation list
- Each index is an integer
- Each integer is a student in the class

2. Create the initial population
- First, a function to create a single chromsome
  - inputs chromosomeSize
  - outputs a chromosome of randomly shuffled integers

7. Testing
- A func